<a href="https://colab.research.google.com/github/SuperDataWorld/Python/blob/main/Imbalanced_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imbalanced Dataset - Sampling Techniques

In [417]:
import pandas as pd #Data manipulation lib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import RandomOverSampler, SMOTE 

In [418]:
#Import Data  
df = pd.read_csv('https://raw.githubusercontent.com/SuperDataWorld/Python/main/Data/loanpred.csv')
df.dropna(axis=1, inplace = True)
df.drop(['ID','Lead_Creation_Date'], axis=1, inplace= True)

# 1. Data Prep

In [419]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69713 entries, 0 to 69712
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           69713 non-null  object 
 1   Monthly_Income   69713 non-null  float64
 2   Contacted        69713 non-null  object 
 3   Source           69713 non-null  object 
 4   Source_Category  69713 non-null  object 
 5   Var1             69713 non-null  int64  
 6   Approved         69713 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 3.7+ MB


In [420]:
df.describe()

,Monthly_Income,Var1,Approved
count,6.971300e+04,69713.000000,69713.000000
mean,5.622283e+03,3.948446,0.014631
std,1.747671e+05,3.819214,0.120073
min,0.000000e+00,0.000000,0.000000
25%,1.650000e+03,0.000000,0.000000
50%,2.500000e+03,2.000000,0.000000
75%,4.000000e+03,7.000000,0.000000
max,3.838384e+07,10.000000,1.000000


In [421]:
df.head()

,Gender,Monthly_Income,Contacted,Source,Source_Category,Var1,Approved
0,Female,2000.0,N,S122,G,0,0
1,Male,3500.0,Y,S122,G,10,0
2,Male,2250.0,Y,S143,B,0,0
3,Male,3500.0,Y,S143,B,7,0
4,Male,10000.0,Y,S134,B,10,0


In [422]:
# Encode Categorical Fields

# Categorical Encoding 
for i in ['Gender','Contacted','Source','Source_Category','Var1']:
  df[i] = df[i].astype('category').cat.codes

In [423]:
X = df.drop(['Approved'], axis=1)
y = df[['Approved']]

In [424]:
y.value_counts()

Approved
0           68693
1            1020
dtype: int64

In [425]:
round(1020/68693,4)

0.0148

#2. Stratified Split

In [426]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)
print(y_train.value_counts())  
print(y_test.value_counts())

Approved
0           54929
1             841
dtype: int64
Approved
0           13764
1             179
dtype: int64


In [427]:
round(841/54929,4)

0.0153

In [428]:
round(179/13764,4)

0.013

In [429]:
splitter=StratifiedShuffleSplit(n_splits=1,random_state=12, test_size=0.20) 
for train,test in splitter.split(X,y):     
    X_train = X.iloc[train]
    y_train = y.iloc[train]
    X_test = X.iloc[test]
    y_test = y.iloc[test]
print(y_train.value_counts())  
print(y_test.value_counts())

Approved
0           54954
1             816
dtype: int64
Approved
0           13739
1             204
dtype: int64


In [430]:
816/54954

0.01484878261818976

In [431]:
204/13739

0.014848242230147754

#2. SMOTE Model

In [432]:
# SMOTE Resample
sm = SMOTE(random_state=42)
X_sm, y_sm = sm.fit_resample(X_train, y_train)

In [433]:
print(y_sm.value_counts()) 

Approved
0           54954
1           54954
dtype: int64


In [434]:
X_sm.tail(10)

,Gender,Monthly_Income,Contacted,Source,Source_Category,Var1
109898,0,4900.000000,0,0,6,2
109899,0,10610.588107,0,11,1,4
109900,0,2800.000000,0,0,6,2
109901,1,4000.000000,1,0,6,3
109902,0,2500.000000,0,6,1,3
109903,0,8000.000000,0,8,1,4
109904,1,4000.000000,1,24,2,3
109905,0,3028.711050,1,19,1,2
109906,0,15000.000000,1,4,2,3
109907,1,5800.000000,1,2,1,3


In [435]:
y_sm.tail(10)

,Approved
109898,1
109899,1
109900,1
109901,1
109902,1
109903,1
109904,1
109905,1
109906,1
109907,1


# 3. Random Oversampling

In [436]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)

In [437]:
print(y_over.value_counts()) 

Approved
0           54954
1           54954
dtype: int64


In [438]:
X_over.tail(10)

,Gender,Monthly_Income,Contacted,Source,Source_Category,Var1
109898,1,2150.0,1,0,2,2
109899,1,13500.0,1,0,6,4
109900,1,5000.0,1,0,1,4
109901,1,2656.8,1,0,6,3
109902,1,4800.0,1,7,1,4
109903,1,5500.0,1,0,6,4
109904,1,4400.0,1,0,6,4
109905,1,3500.0,1,6,4,4
109906,1,3748.9,1,7,2,3
109907,1,6700.0,1,0,6,4


In [439]:
X_over.query('Monthly_Income == 6500 & Contacted == 1 & Source == 6 & Var1 == 4 & Gender == 1 & Source_Category == 1').count()

Gender             244
Monthly_Income     244
Contacted          244
Source             244
Source_Category    244
Var1               244
dtype: int64

In [440]:
X_sm.query('Monthly_Income == 6500 & Contacted == 1 & Source == 6 & Var1 == 4 & Gender == 1 & Source_Category == 1').count()

Gender             151
Monthly_Income     151
Contacted          151
Source             151
Source_Category    151
Var1               151
dtype: int64

In [441]:
X.query('Monthly_Income == 6500 & Contacted == 1 & Source == 6 & Var1 == 4 & Gender == 1 & Source_Category == 1').count()

Gender             48
Monthly_Income     48
Contacted          48
Source             48
Source_Category    48
Var1               48
dtype: int64